# Download audio file


In [10]:

!pip install librosa
import re
import os
from tqdm import tqdm
import pandas as pd
import numpy as np

import subprocess

import scipy.io.wavfile as wav
import librosa

from mega import Mega
import getpass

import warnings; warnings.simplefilter('ignore')

     |████████████████████████████████| 183 kB 7.0 MB/s eta 0:00:01
     |████████████████████████████████| 377 kB 9.8 MB/s eta 0:00:01
     |████████████████████████████████| 323 kB 6.2 MB/s eta 0:00:01
     |████████████████████████████████| 613 kB 23.2 MB/s eta 0:00:01
     |████████████████████████████████| 51 kB 996 kB/s  eta 0:00:01
  Created wheel for librosa: filename=librosa-0.8.0-py3-none-any.whl size=201374 sha256=f65d674c7881ad4b0c38d0efaecdab14d61477c3beb56f6ffd02d0c03d00dd23
  Stored in directory: /Users/andrea/Library/Caches/pip/wheels/aa/5a/92/d52f6f8560ff05a2525e6030a1903412df876714241fb76802
  Created wheel for audioread: filename=audioread-2.1.9-py3-none-any.whl size=23142 sha256=72391334289563b9cfb8b3966d82806ef5c5da4a6d06a04fc4833803c619774a
  Stored in directory: /Users/andrea/Library/Caches/pip/wheels/49/5a/e4/df590783499a992a88de6c0898991d1167453a3196d0d1eeb7
  Created wheel for resampy: filename=resampy-0.2.2-py3-none-any.whl size=320720 sha256=05ab5dd9c3a51ebb

ModuleNotFoundError: No module named 'mega'

## Create a pandas dataset

In [4]:
df=pd.read_csv(os.path.abspath(os.path.join(os.getcwd(),os.pardir,os.pardir))+'/data/data_split.csv',index_col=0)

In [5]:
df.sample(10)

,url,start_time,end_time,class_label
5148,mUkpFO2imUg,440.0,450.0,Humans
11245,3oqAUFCk32s,30.0,40.0,Natural
2980,VY_-w-7pxsI,540.0,550.0,Animal
10315,3pAqTDBgKkw,30.0,40.0,Natural
9660,cTyYvrtKZ74,13.0,23.0,Humans
5004,WCh_NXo1BDs,150.0,160.0,Humans
8253,4LYesVErsjI,30.0,40.0,Humans
8330,tJfVC7vbibE,20.0,30.0,Humans
14599,zdG2zRBh16I,10.0,20.0,Natural
757,lrKc_rot2Kc,0.0,10.0,Animal


In [3]:
df['class_label'].value_counts()

Animal     5000
Natural    5000
Humans     5000
Name: class_label, dtype: int64

In [11]:
?librosa.feature.mfcc

## Download Audio => Extract Features => Upload on Mega


In [6]:
def get_audio_features(filename:str)-> np.array:
    '''
    input: filename (.vaw file)
    output: dictionary containing mffc and mel-spectogram features a
    '''
    hop_length = 512
    
    y, sr = librosa.load(filename)
    
    mfcc_ = librosa.feature.mfcc(y=y, sr=sr, hop_length=hop_length, n_mfcc=13,n_fft=513)
    mel_spect = librosa.feature.melspectrogram(y=y,sr=sr,n_fft=513,win_length=400)
    
    return dict(mfcc = mfcc_, mel_spec = mel_spect)

In [272]:
def save_on_mega(file:str ,m: Mega):
    '''
    save data in folder 'features_'
    the folder has been manually created on Mega website
    '''
    folder = m.find('features_')
    m.upload(file, folder[0])

In [185]:
#login to Mega account 

m = Mega()
email = input('insert email ')
psw = getpass.getpass(prompt='insert password')
m.login(email,psw)

insert email andrea.faviait@gmail.com
insert password········


In [ ]:
not_downloaded = dict(Animal = 0, Humans = 0, Natural = 0)

#n.b. you might have to chance the working directory (os.chdir()))

for i, row in tqdm(df.iterrows()):
    
    url = 'https://www.youtube.com/watch?v=' + row['url']
    file_name = str(i)+"_"+row['class_label']
    
    try:
        
        #download youtube video & create a clipped .wav file
        subprocess.Popen("ffmpeg -ss " + str(row['start_time']) + " -i $(youtube-dl -f 140 --get-url " + url + ") -t 10 -c:v copy -c:a copy " + file_name + ".mp4", shell=True).wait()
        subprocess.Popen("ffmpeg -i "+file_name+".mp4 -ab 160k -ac 2 -ar 44100 -vn "+file_name+'.wav',shell=True).wait()
        
        #extract mfcc, mel features
        res = get_audio_features(file_name+'.wav')
        
        #save .npy file and upload on mega
        file = np.save(file_name,res)
        save_on_mega(file_name+'.npy',m)
        
        #remove .mp4, .wav files, .npy file
        subprocess.Popen('rm '+file_name+'.mp4',shell=True).wait()
        subprocess.Popen('rm '+file_name+'.wav',shell=True).wait()
        subprocess.Popen('rm '+file_name+'.npy',shell=True).wait()


    except Exception as e:
        not_downloaded[row['class_label']] += 1
        pass
    